In [16]:
import os


import argparse
import time
from getpass import getuser
from distutils.dir_util import copy_tree
import pickle

import numpy as np
import random
import theano
import theano.tensor as T
from theano import config
import lasagne
from lasagne.regularization import regularize_network_params
from lasagne.objectives import categorical_crossentropy

from matplotlib import pyplot as plt
from metrics import jaccard, accuracy, crossentropy, weighted_crossentropy
from data_loader.cortical_layers_w_regions_kfold_val_train_test import CorticalLayersDataset

from simple_model_1path import build_simple_model
from profile_functions import profile2indices


In [17]:
_FLOATX = config.floatX
SAVEPATH = '/data1/users/kwagstyl/bigbrain/cortical_layers'
LOADPATH = '/data1/users/kwagstyl/bigbrain/cortical_layers'
WEIGHTS_PATH = LOADPATH


In [18]:
n_filters=64
filter_size = [25]
depth = 8
data_augmentation={}
block = 'bn_relu_conv'
weight_decay=0.001
num_epochs = 500
max_patience =50
resume=False
learning_rate_value = 0.0005
max_clipping=30
prefix='training_40_'
batch_size=[1000,1000,1]
smooth_or_raw = 'both'
shuffle_at_each_epoch = True
minibatches_subset=0
n_layers=6
kfold=10
val_fold=0
test_fold=1

In [19]:
savepath=SAVEPATH
loadpath=LOADPATH
exp_name = 'simple_model'
exp_name += '_dataset' + prefix
exp_name += '_lrate=' + str(learning_rate_value)
exp_name += '_fil=' + str(n_filters)
exp_name += '_fsizes=' + str(filter_size)
exp_name += '_depth=' + str(depth)
exp_name += '_data=' + smooth_or_raw
exp_name += '_decay=' + str(weight_decay)
exp_name += '_pat=' + str(max_patience)
exp_name += '_kfold=' + str(kfold)
exp_name += '_val=' + str(val_fold)
exp_name += '_test=' + str(test_fold)
exp_name += '_batch_size=' + str(batch_size[0])
exp_name += ('_noshuffle'+str(minibatches_subset)+'batch') if not shuffle_at_each_epoch else ''



In [20]:
dataset = str(n_layers)+'cortical_layers'
savepath = os.path.join(savepath, dataset, exp_name)
loadpath = os.path.join(loadpath, dataset, exp_name)
print 'Savepath : '
print savepath
print 'Loadpath : '
print loadpath

if not os.path.exists(savepath):
    os.makedirs(savepath)
else:
    print('\033[93m The following folder already exists {}. '
          'It will be overwritten in a few seconds...\033[0m'.format(
              savepath))

print('Saving directory : ' + savepath)
with open(os.path.join(savepath, "config.txt"), "w") as f:
    for key, value in locals().items():
        f.write('{} = {}\n'.format(key, value))



Savepath : 
/data1/users/kwagstyl/bigbrain/cortical_layers/6cortical_layers/simple_model_datasettraining_40__lrate=0.0005_fil=64_fsizes=[25]_depth=8_data=both_decay=0.001_pat=50_kfold=10_val=0_test=1_batch_size=1000
Loadpath : 
/data1/users/kwagstyl/bigbrain/cortical_layers/6cortical_layers/simple_model_datasettraining_40__lrate=0.0005_fil=64_fsizes=[25]_depth=8_data=both_decay=0.001_pat=50_kfold=10_val=0_test=1_batch_size=1000
Saving directory : /data1/users/kwagstyl/bigbrain/cortical_layers/6cortical_layers/simple_model_datasettraining_40__lrate=0.0005_fil=64_fsizes=[25]_depth=8_data=both_decay=0.001_pat=50_kfold=10_val=0_test=1_batch_size=1000


In [21]:
input_var = T.tensor3('input_var') #n_example*nb_in_channels*ray_size
target_var = T.ivector('target_var') #n_example*ray_size
weight_vector = T.fvector('weight_vector')

learn_step=  theano.shared(np.array(learning_rate_value, dtype=theano.config.floatX))




In [22]:
if smooth_or_raw =='both':
    nb_in_channels = 2
    use_threads = False
else:
    nb_in_channels = 1
    use_threads = True

train_iter = CorticalLayersDataset(
    which_set='train',
    prefix=prefix,
    smooth_or_raw = smooth_or_raw,
    batch_size=batch_size[0],
    return_one_hot=False,
    return_01c=False,
    return_list=False,
    use_threads=use_threads,
    preload=True,
    n_layers=n_layers,
    kfold=kfold, # if None, kfold = number of regions (so there is one fold per region)
    val_fold=val_fold, # it will use the first fold for validation
    test_fold=test_fold) # this fold will not be used to train nor to validate

val_iter = CorticalLayersDataset(
    which_set='valid',
    prefix=prefix,
    smooth_or_raw = smooth_or_raw,
    batch_size=batch_size[1],
    return_one_hot=False,
    return_01c=False,
    return_list=False,
    use_threads=use_threads,
    preload=True,
    n_layers=n_layers,
    kfold=kfold, # if None, kfold = number of regions (so there is one fold per region)
    val_fold=val_fold, # it will use the first fold for validation
    test_fold=test_fold) # this fold will not be used to train nor to validate

test_iter = CorticalLayersDataset(
                                  which_set='test',
                                  prefix=prefix,
                                  smooth_or_raw = smooth_or_raw,
                                  batch_size=batch_size[1],
                                  return_one_hot=False,
                                  return_01c=False,
                                  return_list=False,
                                  use_threads=use_threads,
                                  preload=True,
                                  n_layers=n_layers,
                                  kfold=kfold, # if None, kfold = number of regions (so there is one fold per region)
                                  val_fold=val_fold, # it will use the first fold for validation
                                  test_fold=test_fold) # this fold will not be used to train nor to validate


n_batches_train = train_iter.nbatches
n_batches_val = val_iter.nbatches
n_batches_test = test_iter.nbatches if test_iter is not None else 0
n_classes = train_iter.non_void_nclasses
void_labels = train_iter.void_labels



In [8]:
#
simple_net_output, net = build_simple_model(input_var,
                    filter_size = filter_size,
                    n_filters = n_filters,
                    depth = depth,
                    block= block,
                    nb_in_channels = nb_in_channels,
                    n_classes = n_classes)

#simple_net_output = last layer of the simple_model net


In [9]:
Class=np.loadtxt(os.path.join(LOADPATH,'6layers_segmentation_August2018/training_cls.txt'))
def compute_class_weights(Class):
    #get unique labels and number of pixels of each class
    unique, counts = np.unique(Class,return_counts=True)
    #calculate freq(c) number of pixels per class divided by the total number of pixels in images where c is present
    freq=counts.astype(float)/Class.size
    return np.median(freq)/freq

weights=compute_class_weights(Class)

In [ ]:
def clipping(x_train, l_train, max_clipping = 10):
    """ Clip profiles at either end by random amounts. Augments data"""
    k=-1
    for p, l in zip(x_train, l_train):
        k+=1
        new_p=p.copy()
        cut_top=np.random.randint(max_clipping)
        cut_bottom=np.random.randint(max_clipping)
        if cut_top >0:
            p=np.delete(p,range(cut_top),axis=1)
            l=np.delete(l,range(cut_top))
        if cut_bottom >0:
            l=np.delete(l,p.shape[1]-np.arange(cut_bottom))
            p=np.delete(p,p.shape[1]-np.arange(cut_bottom),axis=1)
            #interpolate back to full length.
        x = np.linspace(0, len(p), x_train.shape[2])
        new_p[0,:]=np.interp(x,range(p.shape[1]),p[0,:])
        new_p[1,:]=np.interp(x,range(p.shape[1]),p[1,:])
        new_label=np.round(np.interp(x,range(p.shape[1]),l))
        x_train[k]=new_p
        l_train[k]=new_label
    return x_train, l_train

# penalty to enforce smoothness
def smooth_convolution(prediction, n_classes):
    from lasagne.layers import Conv1DLayer as ConvLayer
    from lasagne.layers import DimshuffleLayer, ReshapeLayer
    prediction = ReshapeLayer(prediction, (-1, 200, n_classes))
    # channels first
    prediction = DimshuffleLayer(prediction, (0,2,1))

    input_size = lasagne.layers.get_output(prediction).shape
    # reshape to put each channel in the batch dimensions, to filter each
    # channel independently
    prediction = ReshapeLayer(prediction, (T.prod(input_size[0:2]),1,input_size[2]))

    trans_filter = np.tile(np.array([0,-1.,1.]).astype('float32'), (1,1,1))
    convolved = ConvLayer(prediction,
                    num_filters = 1,
                    filter_size = 3,
                    stride=1,
                    b = None,
                    nonlinearity=None,
                    W = trans_filter,
                    pad='same')

    # reshape back
    
    convolved = ReshapeLayer(convolved, input_size)
    return convolved


In [ ]:
print "Defining and compiling training functions"

convolved = smooth_convolution(simple_net_output[0], n_classes)

prediction, convolved = lasagne.layers.get_output([simple_net_output[0], convolved])

#prediction = lasagne.layers.get_output([simple_net_output[0]])
#loss = categorical_crossentropy(prediction, target_var)
#loss = loss.mean()

loss = weighted_crossentropy(prediction, target_var, weight_vector)
loss = loss.mean()


if weight_decay > 0:
    weightsl2 = regularize_network_params(
        simple_net_output, lasagne.regularization.l2)
    loss += weight_decay * weightsl2

#if smooth_penalty > 0:
#    smooth_cost = T.sum(abs(convolved), axis=(1,2))
#   loss += smooth_penalty * smooth_cost.mean()

train_acc, train_sample_acc = accuracy(prediction, target_var, void_labels)

params = lasagne.layers.get_all_params(simple_net_output, trainable=True)
updates = lasagne.updates.adam(loss, params, learning_rate=learn_step)

train_fn = theano.function([input_var, target_var, weight_vector], [loss, train_acc, train_sample_acc, prediction],
                           updates=updates)#, profile=True)

print "Done"
print "Defining and compiling valid functions"
valid_prediction = lasagne.layers.get_output(simple_net_output[0],
                                            deterministic=True)
#valid_loss = categorical_crossentropy(valid_prediction, target_var)
#valid_loss = valid_loss.mean()
valid_loss = weighted_crossentropy(valid_prediction, target_var, weight_vector)
valid_loss = valid_loss.mean()

#valid_loss = crossentropy(valid_prediction, target_var, void_labels)
valid_acc, valid_sample_acc = accuracy(valid_prediction, target_var, void_labels)
valid_jacc = jaccard(valid_prediction, target_var, n_classes)

valid_fn = theano.function([input_var, target_var, weight_vector],
                           [valid_loss, valid_acc, valid_sample_acc, valid_jacc])#,profile=True)
print "Done"
print "Defining and compiling test functions"
test_prediction = lasagne.layers.get_output(simple_net_output[0],
                                            deterministic=True)
test_loss = weighted_crossentropy(valid_prediction, target_var, weight_vector)
test_loss = test_loss.mean()
test_acc, test_sample_acc = accuracy(test_prediction, target_var, void_labels)
test_jacc = jaccard(test_prediction, target_var, n_classes)

test_fn = theano.function([input_var, target_var, weight_vector], [test_loss, test_acc,
                                                    test_sample_acc,test_jacc, test_prediction])
print "Done"



In [ ]:
treshold = 0.7 # for extracting the very incorrect labelled samples
ratios=[0.80,0.85, 0.90] #ratios for the per sample accuracy


# In[ ]:

# Train loop
#
err_train = []
acc_train = []
sample_acc_train_tot = []
worse_indices_train = []
already_seen_idx = []

err_valid = []
acc_valid = []
jacc_valid = []
sample_acc_valid_tot = []
patience = 0
worse_indices_valid =[]

In [ ]:
for epoch in range(num_epochs):
    #learn_step.set_value((learn_step.get_value()*0.99).astype(theano.config.floatX))

    # Single epoch training and validation
    start_time = time.time()
    #Cost train and acc train for this epoch
    cost_train_epoch = 0
    acc_train_epoch = 0
    sample_acc_train_epoch = np.array([0.0 for i in range(len(ratios))])
    # worse_indices_train_epoch = []




    for i in range(n_batches_train):
        print(i)
        # Get minibatch (comment the next line if only 1 minibatch in training)
        train_batch = train_iter.next()
        X_train_batch, L_train_batch = train_batch['data'], train_batch['labels']
   #     if max_clipping > 0:
   #         X_train_batch, L_train_batch = clipping(X_train_batch, L_train_batch, max_clipping)
        L_train_batch = np.reshape(L_train_batch, np.prod(L_train_batch.shape))
        
        # Training step
        cost_train_batch, acc_train_batch, sample_acc_train_batch, pred = train_fn(
                            X_train_batch, L_train_batch, weights[L_train_batch].astype('float32'))
        sample_acc_train_batch_mean = [np.mean([(i>=ratio)
                                for i in sample_acc_train_batch]) for ratio in ratios]

        # worse_indices_train_batch = index_worse_than(sample_acc_train_batch,
        #                                   idx_train_batch, treshold=treshold)

        #print i, 'training batch cost : ', cost_train_batch, ' batch accuracy : ', acc_train_batch

        #Update epoch results
        cost_train_epoch += cost_train_batch
        acc_train_epoch += acc_train_batch
        sample_acc_train_epoch += sample_acc_train_batch_mean
        # worse_indices_train_epoch = np.hstack((worse_indices_train_epoch,worse_indices_train_batch))

    #Add epoch results
    err_train += [cost_train_epoch/n_batches_train]
    acc_train += [acc_train_epoch/n_batches_train]
    sample_acc_train_tot += [sample_acc_train_epoch/n_batches_train]
    # worse_indices_train += [worse_indices_train_epoch]

    # Validation
    cost_val_epoch = 0
    acc_val_epoch = 0
    sample_acc_valid_epoch = np.array([0.0 for i in range(len(ratios))])
    jacc_val_epoch = np.zeros((2, n_classes))
    # worse_indices_val_epoch = []

    for i in range(n_batches_val):
        # Get minibatch (comment the next line if only 1 minibatch in training)
        val_batch = val_iter.next()
        X_val_batch, L_val_batch = val_batch['data'], val_batch['labels']

        L_val_batch = np.reshape(L_val_batch, np.prod(L_val_batch.shape))


        # Validation step
        cost_val_batch, acc_val_batch, sample_acc_valid_batch, jacc_val_batch = valid_fn(X_val_batch, L_val_batch, weights[L_val_batch].astype('float32'))
        #print i, 'validation batch cost : ', cost_val_batch, ' batch accuracy : ', acc_val_batch


        sample_acc_valid_batch_mean = [np.mean([(i>=ratio)
                                for i in sample_acc_valid_batch]) for ratio in ratios]



        #Update epoch results
        cost_val_epoch += cost_val_batch
        acc_val_epoch += acc_val_batch
        sample_acc_valid_epoch += sample_acc_valid_batch_mean
        jacc_val_epoch += jacc_val_batch
        # worse_indices_val_epoch = np.hstack((worse_indices_val_epoch, worse_indices_val_batch))
        #


    #Add epoch results
    err_valid += [cost_val_epoch/n_batches_val]
    acc_valid += [acc_val_epoch/n_batches_val]
    sample_acc_valid_tot += [sample_acc_valid_epoch/n_batches_val]
    jacc_perclass_valid = jacc_val_epoch[0, :] / jacc_val_epoch[1, :]
    jacc_valid += [np.mean(jacc_perclass_valid)]
    # worse_indices_valid += [worse_indices_val_epoch]


    #Print results (once per epoch)

    out_str = "EPOCH %i: Avg cost train %f, acc train %f"+        ", cost val %f, acc val %f, jacc val %f took %f s"
    out_str = out_str % (epoch, err_train[epoch],
                         acc_train[epoch],
                         err_valid[epoch],
                         acc_valid[epoch],
                         jacc_valid[epoch],
                         time.time()-start_time)
    out_str2 = 'Per sample accuracy (ratios ' + str(ratios) + ') '
    out_str2 += ' train ' +str(sample_acc_train_tot[epoch])
    out_str2 += ' valid ' + str(sample_acc_valid_tot[epoch])
    print out_str
    print out_str2

    with open(os.path.join(savepath, "fcn1D_output.log"), "a") as f:
        f.write(out_str + "\n")

    if epoch == 0:
        best_jacc_val = jacc_valid[epoch]
    elif epoch > 1 and jacc_valid[epoch] > best_jacc_val:
        print('saving and testing best (and last) model')
        best_jacc_val = jacc_valid[epoch]
        patience = 0
        np.savez(os.path.join(savepath, 'new_fcn1D_model_best.npz'),
                 *lasagne.layers.get_all_param_values(simple_net_output))
        np.savez(os.path.join(savepath , "fcn1D_errors_best.npz"),
                 err_train=err_train, acc_train=acc_train,
                 err_valid=err_valid, acc_valid=acc_valid, jacc_valid=jacc_valid)
        np.savez(os.path.join(savepath, 'new_fcn1D_model_last.npz'),
                 *lasagne.layers.get_all_param_values(simple_net_output))
        np.savez(os.path.join(savepath , "fcn1D_errors_last.npz"),
                 err_train=err_train, acc_train=acc_train,
                 err_valid=err_valid, acc_valid=acc_valid, jacc_valid=jacc_valid)
        cost_test_epoch=0
        acc_test_epoch=0
        sample_acc_test_epoch=0
        jacc_test_epoch=0
        layer_prediction=[]
                 #test the best iteration and save results for comparisons
        for i in range(n_batches_test):
            test_batch = test_iter.next()
            X_test_batch, L_test_batch = test_batch['data'], test_batch['labels']
        
            L_test_batch = np.reshape(L_test_batch, np.prod(L_test_batch.shape))
        
        # Testing step
            cost_test_batch, acc_test_batch, sample_acc_test_batch, jacc_test_batch, test_pred = test_fn(
                                    X_test_batch, L_test_batch, weights[L_test_batch].astype('float32'))
            for l,p in zip(L_test_batch,test_pred):
                layer_prediction.append(layer_prediction(p,l))
                
            sample_acc_test_batch_mean = [np.mean([(i>=ratio)
                                    for i in sample_acc_test_batch]) for ratio in ratios]
                                    

                                    
            cost_test_epoch += cost_test_batch
            acc_test_epoch += acc_test_batch
            sample_acc_test_epoch += sample_acc_test_batch_mean
            jacc_test_epoch += jacc_test_batch
        layer_prediction=np.array(layer_prediction)
        layer_std=np.std(layer_prediction,axis=0)
        layer_mean=np.mean(layer_prediction,axis=0)
        err_test = [cost_test_epoch/n_batches_test]
        acc_test = [acc_test_epoch/n_batches_test]
        sample_acc_test_tot = [sample_acc_test_epoch/n_batches_test]
        jacc_perclass_test = jacc_test_epoch[0, :] / jacc_test_epoch[1, :]
        jacc_test = [np.mean(jacc_perclass_test)]

        np.savez(os.path.join(savepath , "fcn1D_test_errors_best.npz"),
             err_test=err_test, acc_test=acc_test, jacc_test=jacc_test, layer_std=layer_std, layer_mean=layer_mean)
    else:
        patience += 1
        print('saving last model')
        np.savez(os.path.join(savepath, 'new_fcn1D_model_last.npz'),
                 *lasagne.layers.get_all_param_values(simple_net_output))
        np.savez(os.path.join(savepath , "fcn1D_errors_last.npz"),
                 err_train=err_train, acc_train=acc_train,
                 err_valid=err_valid, acc_valid=acc_valid, jacc_valid=jacc_valid)
    # Finish training if patience has expired or max nber of epochs reached

    if patience == max_patience or epoch == num_epochs-1:
        if savepath != loadpath:
            print('Copying model and other training files to {}'.format(loadpath))
            copy_tree(savepath, loadpath)
        break





In [14]:
from theano import function, config, shared, tensor
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[Elemwise{exp,no_inplace}(<TensorType(float32, vector)>)]
Looping 1000 times took 3.286502 seconds
Result is [1.2317803 1.6187934 1.5227807 ... 2.2077181 2.2996776 1.6232328]
Used the cpu


In [18]:
acc_val_batch

array(0.22074766, dtype=float32)

In [19]:
acc_val_epoch

1.1113976687192917